In [1]:
import pandas as pd

# Creamos el dataframe (ajustar la ruta según tus directorios)
df = pd.read_csv(r"C:\Users\Yago\Documents\Basketmática\Olympics_Julio2024\basketball_olympic.csv")
df.head()

,year,host,winner,runner-up,round,winner_points,runner-up_points
0,1936,Berlin,United States,Canada,final,19,8
1,1936,Berlin,Mexico,Poland,third place game,26,12
2,1948,London,United States,France,final,65,21
3,1948,London,Brazil,Mexico,third place game,52,47
4,1952,Helsinki,United States,Soviet Union,final,36,25


# Preprocesado de datos

In [2]:
# Primero, comprobamos si existe algún valor nulo dentro de nuestro conjunto de datos
df.isnull().sum()

year                0
host                0
winner              0
runner-up           0
round               0
winner_points       0
runner-up_points    0
dtype: int64

Explorando los datos, detectamos una pequeña incongruencia, ya que en Atlanta 1996 'Yugoslavia' figuraba como 'FR Yugoslavia'. Se corrige rápidamente dicho valor

In [3]:
df.loc[26, 'runner-up'] = 'Yugoslavia'

Creamos un dataframe para los partidos correspondientes a finales y otro para los partidos de tercer y cuarto puesto

In [4]:
# Creamos los dataframes para uso posterior
df_finals = df[df['round'] == 'final']
df_thirdplace = df[df['round'] == 'third place game']
df_finals.head()

,year,host,winner,runner-up,round,winner_points,runner-up_points
0,1936,Berlin,United States,Canada,final,19,8
2,1948,London,United States,France,final,65,21
4,1952,Helsinki,United States,Soviet Union,final,36,25
6,1956,Melbourne,United States,Soviet Union,final,89,55
8,1960,Rome,United States,Soviet Union,final,81,57


# 1. Recuento de medallas

In [5]:
# 1. Equipos que más veces ha sido oro
print(df_finals['winner'].value_counts())

United States    16
Soviet Union      2
Yugoslavia        1
Argentina         1
Name: winner, dtype: int64


In [6]:
# 2. Equipos que más veces ha sido plata
print(df_finals['runner-up'].value_counts())

Soviet Union     4
Yugoslavia       4
France           3
Spain            3
Italy            2
Canada           1
United States    1
Croatia          1
Serbia           1
Name: runner-up, dtype: int64


In [7]:
# 3. Equipos que más veces ha sido bronce
print(df_thirdplace['winner'].value_counts())

Brazil           3
Soviet Union     3
Lithuania        3
Uruguay          2
United States    2
Mexico           1
Cuba             1
Yugoslavia       1
Argentina        1
Russia           1
Spain            1
Australia        1
Name: winner, dtype: int64


# 2. Análisis de puntuación en los partidos por medalla

In [8]:
# 1. Media de puntos anotados en la final
media_puntos_ganador = df_finals['winner_points'].mean()
media_puntos_perdedor = df_finals['runner-up_points'].mean()
print(f"Media de puntos equipo ganador: {media_puntos_ganador:.2f}")
print(f"Media de puntos equipo perdedor: {media_puntos_perdedor:.2f}")

Media de puntos equipo ganador: 81.05
Media de puntos equipo perdedor: 62.85


In [9]:
# 2. Media de puntos anotados en la lucha por el bronce
media_puntos_ganador2 = df_thirdplace['winner_points'].mean()
media_puntos_perdedor2 = df_thirdplace['runner-up_points'].mean()
print(f"Media de puntos equipo ganador: {media_puntos_ganador2:.2f}")
print(f"Media de puntos equipo perdedor: {media_puntos_perdedor2:.2f}")

Media de puntos equipo ganador: 80.45
Media de puntos equipo perdedor: 69.10


In [10]:
# 3. Partido más anotador luchando por medalla
df['total_points'] = df['winner_points'] + df['runner-up_points']
max_points_row = df.loc[df['total_points'].idxmax()]
print(f"Total puntos anotados: {max_points_row['total_points']}")
print(f"Año: {max_points_row['year']}  Sede: {max_points_row['host']}")
print(f"Ganador: {max_points_row['winner']}")
print(f"Perdedor: {max_points_row['runner-up']}")

Total puntos anotados: 225
Año: 2008  Sede: Beijing
Ganador: United States
Perdedor: Spain


In [11]:
# 4. Partido menos anotador luchando por medalla
min_points_row = df.loc[df['total_points'].idxmin()]
print(f"Total puntos anotados: {min_points_row['total_points']}")
print(f"Año: {min_points_row['year']}  Sede: {min_points_row['host']}")
print(f"Ganador: {min_points_row['winner']}")
print(f"Perdedor: {min_points_row['runner-up']}")

Total puntos anotados: 27
Año: 1936  Sede: Berlin
Ganador: United States
Perdedor: Canada


# 3. Anfitriones con medalla

In [12]:
# Creamos un diccionario con la lista de sedes y la selección a la que corresponde
lista_sedes = {'Berlín': 'Germany',
              'London': 'United Kingdom',
              'Helsinki': 'Finland',
              'Melbourne': 'Australia',
              'Rome': 'Italy',
              'Tokyo': 'Japan',
              'Mexico City': 'Mexico',
              'Munich': 'Germany',
              'Montreal': 'Canada',
              'Moscow': 'Soviet Union',
              'Los Angeles': 'United States',
              'Seoul': 'South Korea',
              'Barcelona': 'Spain',
              'Atlanta': 'United States',
              'Sydney': 'Australia',
              'Athens': 'Greece',
              'Beijing': 'China',
              'London': 'United Kingdom',
              'Rio de Janeiro': 'Brazil',
              'Tokyo': 'Japan'}

for index, row in df_finals.iterrows():
    # Obtenemos la ciudad donde se celebró los JJOO y el ganador para cada edición
    city, country = row['host'], row['winner']
    # Revisamos si la sede coincide con el mapeo de la selección ganadora
    if city in lista_sedes and lista_sedes[city] == country:
        print(f"Oro: {row['winner']} | Ciudad: {city} | Año: {row['year']}")
        print('\n')
    # O si coincide con el subcampeón de dicha edición
    country = row['runner-up']
    if city in lista_sedes and lista_sedes[city] == country:
        print(f"Plata: {row['runner-up']} | Ciudad: {city} | Año: {row['year']}")
        print('==========')

for index, row in df_thirdplace.iterrows():
    # Repetimos el proceso anterior con los partidos de tercer y cuarto puesto, pero solo es necesario revisar el ganador (bronce)
    city, country = row['host'], row['winner']
    if city in lista_sedes and lista_sedes[city] == country:
        print(f"Bronce: {row['winner']} | Ciudad: {city} | Año: {row['year']}")

Oro: United States | Ciudad: Los Angeles | Año: 1984


Oro: United States | Ciudad: Atlanta | Año: 1996


Bronce: Soviet Union | Ciudad: Moscow | Año: 1980


In [19]:
# Guardo el dataframe actualizado en formato Excel para importarlos en Tableau y crear las visualizaciones (ajustar la ruta según tus directorios)
df.to_excel(r"C:\Users\Yago\Documents\Basketmática\Olympics_Julio2024\basketball_olympic_cleaned.xlsx")